In [132]:
import pandas as pd

messages = pd.read_csv('SMSSpamCollection.txt', sep='\t',
                           names=["label", "message"])

In [133]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [134]:
messages.isnull().sum()

label      0
message    0
dtype: int64

In [135]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(messages['message'], messages['label'], test_size=0.2, random_state=42)

In [136]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(4457,) (1115,) (4457,) (1115,)


In [137]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [138]:
import re
from nltk.corpus import stopwords

corpus = []
for i in x_train:
    review = re.sub('[^a-zA-Z]', ' ', i)
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [9]:
%pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.0 MB 504.4 kB/s eta 0:00:48
   ---------------------------------------- 0.1/24.0 MB 504.4 kB/s eta 0:00:48
   ---------------------------------------- 0.2/24.0 MB 437.1 kB/s eta 0:00:55
   ---------------------------------------- 0.2/24.0 MB 437.1 kB/s eta 0:00:55
   ---------------------------------------- 0.2/24.0 MB 378.3 kB/s eta 0:01:04
   ---------------------------------------- 0.2/24.0 MB 393.0 kB/s eta 0:01:01
   ---------------------------------------- 0.2/24.0 MB 393.0 kB/s eta 0:01:01
   ---------------------------------------- 0.2/24.0 MB 393.0 kB/s eta 0:01:01
   ---------------------------------------- 0.2/24.0 MB 344.8 kB/s eta 0:01:10
   ---------------------------------------- 0.2/24.0 MB 344.8 kB/s eta 0:01:10


In [10]:
from nltk import sent_tokenize
import gensim
from gensim.utils import simple_preprocess

In [139]:
corpus[0:2]

['reply win weekly fifa world cup held send stop end service',
 'hello sort town already dont rush home eating nacho let know eta']

In [140]:
len(corpus)

4457

In [141]:
indices = []
for i in range(len(corpus)):
    if corpus[i] == "" or corpus[i] == " ":
        indices.append(i)

In [142]:
len(corpus)

4457

In [143]:
y_train = list(y_train)

y_train_filtered = [y for i, y in enumerate(y_train) if i not in indices]
corpus_filtered = [sent for i, sent in enumerate(corpus) if i not in indices]

In [144]:
words=[]
i = 0
c1,c2 = [],[]
for sent in corpus_filtered:
    sent_token=sent_tokenize(sent)
    for word in sent_token:
        i += 1
        c1.append(i)
        token = simple_preprocess(word)
        if token:
            c2.append(i)
            words.append(token)
        else:
            words.append(' ')

In [145]:
len(c1), len(c2), len(words)

(4450, 4448, 4450)

In [146]:
words[:2]

[['reply',
  'win',
  'weekly',
  'fifa',
  'world',
  'cup',
  'held',
  'send',
  'stop',
  'end',
  'service'],
 ['hello',
  'sort',
  'town',
  'already',
  'dont',
  'rush',
  'home',
  'eating',
  'nacho',
  'let',
  'know',
  'eta']]

In [147]:
len(words)

4450

In [148]:
print(corpus[-1])
print(words[-1])

say somebody named tampa
['say', 'somebody', 'named', 'tampa']


In [149]:
model = gensim.models.Word2Vec(words, window=5)

In [150]:
model.corpus_count

4450

In [18]:
import numpy as np

def avgWord2Vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis=0)

In [19]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [20]:
from tqdm import tqdm

In [151]:
X = []
for i in tqdm(range(len(words))):
    X.append(avgWord2Vec(words[i]))

  0%|          | 0/4450 [00:00<?, ?it/s]c:\Users\karat\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\karat\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 4450/4450 [00:02<00:00, 2117.55it/s]


In [152]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

forest = RandomForestClassifier(n_estimators=100, random_state=42)

In [158]:
len(X),len(y_train_filtered)

(4450, 4450)

In [161]:
y_train_filtered = [1 if i == 'spam' else 0 for i in y_train_filtered]

In [162]:
type(X[0]), type(y_train_filtered[0])

(numpy.ndarray, int)

In [164]:
for i in range(len(X)):
    if type(X[i]) != np.ndarray:
        print(i)

32
42
121
221
297
316
353
398
401
423
512
525
626
724
739
765
849
864
888
910
968
1154
1192
1263
1315
1491
1505
1635
1667
1782
1855
1991
2096
2101
2121
2210
2255
2297
2332
2454
2472
2535
2613
2850
2911
2934
2961
2997
3024
3041
3089
3161
3446
3479
3748
3770
3807
4049
4073
4130
4193
4397


In [166]:
for i in range(len(y_train_filtered)):
    if type(y_train_filtered[i]) != int:
        print(i)

In [169]:
X[0].shape

(100,)

In [170]:
for i in range(len(X)):
    if X[i].shape != (100,):
        X[i] = np.zeros(shape = (100,))

In [171]:
for i in range(len(X)):
    if type(X[i]) != np.ndarray:
        print(i)

In [172]:
forest.fit(X, y_train_filtered)

RandomForestClassifier(random_state=42)

In [ ]:
def pipeline(text):
    corpus = []
    for i in text:
        review = re.sub('[^a-zA-Z]', ' ', i)
        review = review.lower()
        review = review.split()
        
        review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
        words=[]
    for sent in corpus:
        sent_token=sent_tokenize(sent)
        for word in sent_token:
            token = simple_preprocess(word)
            if token:
                words.append(token)
            else:
                words.append(' ')
    model = gensim.models.Word2Vec(words, window=5)
    def avgWord2Vec(doc):
        return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis=0)
    X = []
    for i in range(len(words)):
        X.append(avgWord2Vec(words[i]))
    for i in range(len(X)):
        if X[i].shape != (100,):
            X[i] = np.zeros(shape = (100,))
    return X

In [ ]:
def document_to_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)  
    return np.mean(vectors, axis=0)

X_test = np.array([document_to_vector(doc, model) for doc in test_docs])

In [ ]:
forest.score(X, y_train_filtered)